In [1]:
import json
import pandas as pd
import numpy as np
import requests

ACCOUNT_ID = "001-001-2265866-001"
API_TOKEN = "591691d91a571232b46b9b87e74145ba-4fc5bc3427b4eb7d79c3501316fdffa0"

API_URL = "api-fxtrade.oanda.com"
STREAM_URL = "stream-fxtrade.oanda.com"

PRICING_PATH = f"/v3/accounts/{ACCOUNT_ID}/pricing"

query = {"instruments": "EUR_USD"} 
headers = {"Authorization": "Bearer "+ API_TOKEN}

response = requests.get("https://"+API_URL+PRICING_PATH, headers=headers, params=query) 

response.json()

{'time': '2024-08-03T20:00:00.065448339Z',
 'prices': [{'type': 'PRICE',
   'time': '2024-08-02T20:59:05.204448944Z',
   'bids': [{'price': '1.09101', 'liquidity': 500000},
    {'price': '1.09100', 'liquidity': 2500000},
    {'price': '1.09099', 'liquidity': 2000000},
    {'price': '1.09098', 'liquidity': 5000000},
    {'price': '1.09095', 'liquidity': 10000000},
    {'price': '1.09092', 'liquidity': 10000000}],
   'asks': [{'price': '1.09121', 'liquidity': 500000},
    {'price': '1.09122', 'liquidity': 500000},
    {'price': '1.09123', 'liquidity': 2000000},
    {'price': '1.09124', 'liquidity': 2000000},
    {'price': '1.09125', 'liquidity': 5000000},
    {'price': '1.09127', 'liquidity': 10000000},
    {'price': '1.09130', 'liquidity': 10000000}],
   'closeoutBid': '1.09092',
   'closeoutAsk': '1.09130',
   'status': 'non-tradeable',
   'tradeable': False,
   'quoteHomeConversionFactors': {'positiveUnits': '1.00000000',
    'negativeUnits': '1.00000000'},
   'instrument': 'EUR_USD'}

In [12]:
import time

from_time = time.mktime(pd.to_datetime("01/01/2019").timetuple())
to_time = time.mktime(pd.to_datetime("01/02/2019").timetuple())


print(f"Window start: {from_time}, window end: {to_time}." )

header = {"Authorization": "Bearer "+API_TOKEN} 
query = {"from": str(from_time), "to": str(to_time), "granularity": "M1"}


INSTRUMENT = "GBP_JPY"

CANDLES_PATH = f"/v3/accounts/{ACCOUNT_ID}/instruments/{INSTRUMENT}/candles"


response = requests.get("https://"+API_URL+CANDLES_PATH, headers=header, params=query)


def json_to_pandas(json):
    json_file = json.json()
    price_json = json_file["candles"]
    dateTimes = []
    close_price, high_price, low_price, open_price= [], [], [], []
    volumes = []

    for candle in price_json:
        volumes.append(candle["volume"])
        dateTimes.append(candle["time"])
        close_price.append(float(candle["mid"]["c"]))
        high_price.append(float(candle["mid"]["h"]))
        low_price.append(float(candle["mid"]["l"]))
        open_price.append(float(candle["mid"]["o"]))
        
    dataFrame = pd.DataFrame({"DateTime": dateTimes,
                              "Open": open_price,"High": high_price,
                              "Low": low_price,"Close": close_price,
                              "Volume": volumes})
    dataFrame['DateTime'] = pd.to_datetime(dataFrame['DateTime'])
    dataFrame.set_index('DateTime', inplace=True)

    return dataFrame

GBPJPY_df = json_to_pandas(response)

GBPJPY_df.head()

AttributeError: 'str' object has no attribute 'timetuple'

In [3]:
from OandaAPI import OandaAPI
oa = OandaAPI(account_id=ACCOUNT_ID, api_token=API_TOKEN)

gbp_jpy_df = oa.show_granularity()

gbp_jpy_df

,Description
Granularity Value,
S5,"5 second candlesticks, minute alignment"
S10,"10 second candlesticks, minute alignment"
S15,"15 second candlesticks, minute alignment"
S30,"30 second candlesticks, minute alignment"
M1,"1 minute candlesticks, minute alignment"
M2,"2 minute candlesticks, hour alignment"
M4,"4 minute candlesticks, hour alignment"
M5,"5 minute candlesticks, hour alignment"
M10,"10 minute candlesticks, hour alignment"


In [4]:
df = oa.get_historical_data(instrument='EUR_USD', 
                            start_date='08/09/2020',
                            end_date='08/09/2021',
                            granularity='H8')
df

,Open,High,Low,Close,Volume
DateTime,,,,,
2020-08-09 21:00:00+00:00,1.17934,1.17984,1.17737,1.17910,6627
2020-08-10 05:00:00+00:00,1.17912,1.18008,1.17406,1.17676,43054
2020-08-10 13:00:00+00:00,1.17678,1.17920,1.17362,1.17368,36857
2020-08-10 21:00:00+00:00,1.17376,1.17552,1.17221,1.17524,7778
2020-08-11 05:00:00+00:00,1.17522,1.18081,1.17226,1.17976,55632
...,...,...,...,...,...
2021-08-05 13:00:00+00:00,1.18360,1.18514,1.18308,1.18324,27011
2021-08-05 21:00:00+00:00,1.18346,1.18359,1.18186,1.18255,4597
2021-08-06 05:00:00+00:00,1.18256,1.18299,1.17764,1.17844,38433


In [7]:
from datetime import datetime
then = datetime(2012, 3, 5, 23, 8, 15)        # Random date in the past
now  = datetime.now()                         # Now
duration = now - then                         # For build-in functions
duration_in_s = duration.total_seconds() 

In [10]:

gran_secs = {'S5': 5,'S10': 10,'S15': 15,'S30': 30,
             'M1': 60,'M2': 120,'M4': 240,'M5': 300,
             'M10': 600,'M15': 900,'M30': 1800,
             'H1': 3600,'H2': 7200,'H3': 10800,
             'H4': 14400,'H6': 21600,'H8': 28800,
             'H12': 43200,'D': 86400,'W': 604800,
             'M': 2620800}

duration_in_s/gran_secs.get('H12')

9067.769245436435

In [14]:
from datetime import timedelta
what = now-timedelta(seconds=5000*gran_secs.get('M1'))



datetime.datetime(2024, 7, 31, 9, 2, 6, 402854)